In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
!pip install tensorflow-gpu==2.0.0b0 --user


  Using cached https://files.pythonhosted.org/packages/e8/7e/87c4c94686cda7066f52cbca4c344248516490acdd6b258ec6b8a805d956/tensorflow_gpu-2.0.0b0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: tensorflow-gpu 1.14.0
    Uninstalling tensorflow-gpu-1.14.0:
      Successfully uninstalled tensorflow-gpu-1.14.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install np_utils --user

You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install keras==2.2.4 --user
 

You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!pip install gensim --user

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

import nltk

In [7]:
!pip install keras==2.2.4 --user

You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import tensorflow.keras  as keras

In [9]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from  keras.preprocessing.text import Tokenizer
from  keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

Using TensorFlow backend.


In [10]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [11]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [12]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kuadmin01/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [13]:
word_files = ['naturetravel.txt']
word_dishes = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_dishes.extend(word_list2)
word_dishes[0:5]

['view', 'views', 'sunset', 'seaside', 'sea']

In [14]:
word_files = ['shoppingtravel.txt']
word_restaurant = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_restaurant.extend(word_list2)
word_restaurant[0:5]

['street', 'markets', 'street food', 'streetfood', 'shop']

In [15]:
#word_files = ['shopping&market.txt']
#word_location = []
#for word_file in word_files:
#    with open(word_file) as f:
#        lines = f.readlines()
#    lists_l= [x.strip() for x in lines] 
    
#    word_location.extend(lists_l)
#word_location[0:5]

In [16]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [17]:
word_dishes = ordered_set(word_dishes)

In [18]:
word_restaurant = ordered_set(word_restaurant)

In [19]:
#word_location = ordered_set(word_location)

In [20]:
mw_list = []
for x in word_dishes:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

for x in word_restaurant:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

        
#for x in word_location:
#    ws = x.split() 
#    if len(ws) > 1:
#        tws = tuple(ws)
#        mw_list.append(tws)
        
mw_list = ordered_set(mw_list)
mw_list[0:5]

[('street', 'food'), ('walking', 'street'), ('food', 'court')]

In [21]:
my_idx = {}
for i,w in enumerate(mw_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

3


In [22]:
for i in mw_list:
    if i == ('friendly','service'):
        print('found')
    if i == ('asian','food'):
        print('found')

In [23]:
mwe = MWETokenizer(mw_list,separator=' ')

In [24]:
reviews = pd.read_csv("BERT-ShopTours-NER.csv")

In [25]:
reviews.shape

(8894, 1)

In [26]:
reviews.head()

,summary
0,000 bhat on a wooden standing lanna buddha whi...
1,10 mins drive and we arrived at the elephant h...
2,10 yrs and 12 yrs did not really expect anythi...
3,100 503920 e 13 45 12 5 n 100 30 14 1 e within...
4,100 recommended place they really care about t...


In [27]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [28]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt.lower()))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [29]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [30]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        # We are not using "text.split()" here
        #since it is not fool proof, e.g. words followed by punctuations "Are you kidding?I think you aren't."
        text = re.findall(r"[\w']+", text)
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)# remove links
    text = re.sub(r'\<a href', ' ', text)# remove html link tag
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.!?:#$\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [31]:
clean_restaurant = []
for restaurants in reviews.summary:
    clean_restaurant.append(clean_text(restaurants, remove_stopwords=False))
print("restaurants are complete.")

restaurants are complete.


In [33]:
from tqdm import tqdm_notebook as tqdm
all_item = []

for i in tqdm(range(len(reviews))):
    word_ls = []
    #print("sentence:",i+1)
    
    for token in spans(clean_restaurant[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        my_tuple = token[0]
        #print("###",token)
        
        #my_tuples = ' , '.join(map(str, my_tuple))
        if token[0] in word_dishes:
            #word_ls.append(my_tuple)
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-NTV']*len(pos_list)
            tag_list[0] = 'B-NTV' 
            
            
            for s,p,t in zip(subwords,pos_list,tag_list):           
                if type(p) == list:
                    p = p[0][1]
                    #print('list')
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
                
                
            #lis_lo = nltk.pos_tag(word_ls),LOC
            #print(' , '.join(map(str, lis_lo)))
 
        elif token[0] in word_restaurant:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-STV']*len(pos_list)
            tag_list[0] = 'B-STV' 
            
            #print(subwords)
            for s,p,t in zip(subwords,pos_list,tag_list):           
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p[0][1]})
                all_item.append(new_item)
                #print(new_item)
            #word_ls.append(my_tuple)
            #print("found LOC")
            #lis_lo = nltk.pos_tag(word_ls),"LOC"
            #print(' , '.join(map(str, lis_lo)))
             
#        elif token[0] in word_location:
            #word_ls.append(my_tuple)
            #lis_lo = nltk.pos_tag(word_ls),"FACILITY"
            #print(' , '.join(map(str, lis_lo)))
            
            #my_pos = nltk.pos_tag([my_tuple])[0][1]
            #new_item =  dict({'Sentence #': i+1, 'Tag' : 'FACILITY', 'Word': my_tuple,'POS': my_pos})
            
#            subwords = token[0].split()
#            pos_list = [nltk.pos_tag([w]) for w in subwords]
#            tag_list = ['I-SHOP']*len(pos_list)
#            tag_list[0] = 'B-SHOP' 
            
             
#            for s,p,t in zip(subwords,pos_list,tag_list):   
#                if type(p) == list:
#                    p = p[0][1]
#                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
#                all_item.append(new_item)
                

             
        else:
            #print(type(my_tuple))
            my_pos = nltk.pos_tag([my_tuple])[0][1]
            new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
            all_item.append(new_item)
            
            #if i+1 == 5177:
                #print(new_item)
             
            #print(nltk.pos_tag([my_tuple]),',','O')
        #print(new_item)
        #if not(my_pos == '.' or  my_pos == ',' or my_pos == ':' or my_pos == '(' or my_pos == ')') :
            
            #all_item.append(new_item) 

file_csv = 'ShopTour-bert.csv'
with open(file_csv, 'w') as csv_file:
    csv_file.write('Sentence #*Word*POS*Tag\n')
    for item in all_item:      
        #print(item['POS'])    
        csv_file.write(str(item['Sentence #'])+'*'+item['Word']+'*'+item['POS']+'*'+item['Tag']+'\n')
csv_file.close()

In [34]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
#load data
# explore data
file_csv = 'ShopTour-bert.csv'
data = pd.read_csv(file_csv, sep='*',encoding="utf-8").fillna(method="ffill")

data.head(10)

,Sentence #,Word,POS,Tag
0,1,000,CD,O
1,1,bhat,IN,O
2,1,on,IN,O
3,1,a,DT,O
4,1,wooden,NN,O
5,1,standing,VBG,O
6,1,lanna,NN,O
7,1,buddha,NN,B-NTV
8,1,which,WDT,O
9,1,the,DT,O


In [35]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [36]:
#concat sentence
getter = SentenceGetter(data)

In [37]:
word_list = [ [s[0] for s in sent] for sent in getter.sentences] 

In [38]:
sentences = word_list
sentences[0]

['000',
 'bhat',
 'on',
 'a',
 'wooden',
 'standing',
 'lanna',
 'buddha',
 'which',
 'the',
 'owner',
 'actually']

In [39]:
print(len(sentences))

8894


In [40]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])
print(len(labels[0]))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NTV', 'O', 'O', 'O', 'O']
12


In [41]:
print(len(word_list[0]))

12


In [42]:
print(len(labels))

8894


In [43]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals) }

In [44]:
words = list(set(data["Word"].values))
n_words = len(words); 
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
n_words

11413

In [45]:
idx2tag[0]

'B-STV'

In [46]:
import sys
sys.path.append('pytorch-pretrained-BERT/')

In [47]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [48]:
MAX_LEN = 128
bs = 32
device = torch.device("cuda")

n_gpu = torch.cuda.device_count()
#torch.cuda.get_device_name(0)
torch.cuda.set_device(1)

In [49]:
tokenized_texts = word_list
print(tokenized_texts[0])

['000', 'bhat', 'on', 'a', 'wooden', 'standing', 'lanna', 'buddha', 'which', 'the', 'owner', 'actually']


In [50]:
sentences[0]

['000',
 'bhat',
 'on',
 'a',
 'wooden',
 'standing',
 'lanna',
 'buddha',
 'which',
 'the',
 'owner',
 'actually']

In [51]:
tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]
print(len(tokens_ids[0]),len(labels[0]))

12 12


In [52]:
 print(len(tokens_ids[0]),len(labels[0]))

12 12


In [53]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
input_ids = pad_sequences(tokens_ids,
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")
#input_ids = pad_sequences([convert_word2idx(txt) for txt in tokenized_texts],
                          #maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")

In [54]:
print(MAX_LEN)
for i in tokens_ids:
    if len(i) > MAX_LEN:
         
        i = i[:128]
        print("need more")
         
        #MAX_LEN = len(i)
print(MAX_LEN)

128
128


In [55]:
t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
len(t_list[0])

12

In [56]:
print(input_ids[0])

[ 4676  3385  5620  6739  7279 10233 10035  4846 10862  8807 10300  8138
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [57]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")

In [58]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [59]:
#split train test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [60]:
#change to torhc tensor
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [61]:
print(tr_inputs.shape)
print(tr_masks.shape)
print(tr_tags.shape)

torch.Size([8004, 128])
torch.Size([8004, 128])
torch.Size([8004, 128])


In [62]:
print(val_inputs.shape)
print(val_masks.shape)
print(val_tags.shape)

torch.Size([890, 128])
torch.Size([890, 128])
torch.Size([890, 128])


In [63]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [64]:
print(len(tag2idx))

4


In [65]:
model = BertForTokenClassification.from_pretrained(u"bert-large-uncased", num_labels=len(tag2idx))

In [66]:
model.cuda();

In [67]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [68]:
#Fine tune BERT
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [69]:
torch.cuda.set_device(1)

In [70]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels, true_inputs = [], [],[]
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        inputs = b_input_ids.to('cpu').numpy()
         
        true_inputs.append(inputs)
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]


    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.13749892727854066
Validation loss: 0.03210216985150639
Validation Accuracy: 0.9861617874313187


Epoch:  10%|█         | 1/10 [03:40<33:08, 220.94s/it]

F1-Score: 0.6075638506876228
Train loss: 0.015150981430549628
Validation loss: 0.010620615900864192
Validation Accuracy: 0.9953680352850275


Epoch:  20%|██        | 2/10 [07:21<29:27, 220.88s/it]

...

F1-Score: 0.9899062725306416
Train loss: 0.0029245113663243076
Validation loss: 0.0032410668554803124
Validation Accuracy: 0.9997471400669643


Epoch:  90%|█████████ | 9/10 [33:07<03:40, 220.74s/it]

F1-Score: 0.988555078683834
Train loss: 0.0009714847630233584
Validation loss: 0.0029831355813030314
Validation Accuracy: 0.9986397879464286


Epoch: 100%|██████████| 10/10 [36:47<00:00, 220.70s/it]

F1-Score: 0.946103673189152
